In [320]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

In [498]:
import os
import numpy as np
from keras.layers import Dropout
import datetime
import time

In [322]:

import re
import functools


import cv2
import tensorflow as tf

from PIL import Image
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from collections import Counter

import string

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.preprocessing.image import load_img
from Encoder import Encoder

In [323]:
train = pd.read_csv('./train.csv')  
test = pd.read_csv('./test.csv') 
data = pd.read_csv('./input.csv')
dataset = data[["Image1","Image2","Clean_capt","Number"]]
dataset.head()


,Image1,Image2,Clean_capt,Number
0,CXR162_IM-0401-1001,CXR162_IM-0401-2001,normal chest,1
1,CXR1390_IM-0249-1001,CXR1390_IM-0249-2001,no evidence active disease,2
2,CXR604_IM-2193-1001,CXR604_IM-2193-2001,no evidence active disease,3
3,CXR2699_IM-1167-1001,CXR2699_IM-1167-2001,no acute cardiopulmonary disease,4
4,CXR2841_IM-1253-2001,CXR2841_IM-1253-2001,no acute cardiopulmonary disease,5


In [324]:
dataset.iloc[0:3,]

,Image1,Image2,Clean_capt,Number
0,CXR162_IM-0401-1001,CXR162_IM-0401-2001,normal chest,1
1,CXR1390_IM-0249-1001,CXR1390_IM-0249-2001,no evidence active disease,2
2,CXR604_IM-2193-1001,CXR604_IM-2193-2001,no evidence active disease,3


In [416]:
def get_img_features(df, input_size):
    path = '/Users/shuyuexu/Documents/COLUMBIA/2021fall/4995AML/project/NLMCXR_png'
    
    Xnet_features = {}
    for i in range(len(df)):
        f = os.path.join(path, df.iloc[i]["Image1"].lstrip())
        f = f+".png"
        image1 = Image.open(f)
        image1 = image1.resize((input_size[0], input_size[1]),Image.NEAREST)
        image1 = np.array(image1).reshape(1,input_size[0],input_size[1],input_size[2])/255
        
        f = os.path.join(path, df.iloc[i]["Image2"].lstrip())
        f = f+".png"
        
        image2 = Image.open(f)
        image2 = image2.resize((input_size[0], input_size[1]),Image.NEAREST)
        image2 = np.array(image2).reshape(1,input_size[0],input_size[1],input_size[2])/255

        en_model = Encoder()
        en_model.build(input_shape = (None, input_size[0],input_size[1],input_size[2]))

        image1_features = en_model.predict(image1)
        image2_features = en_model.predict(image2)
        input_concat = np.concatenate((image1_features, image2_features), axis=1)
        
        Xnet_features[int(df.iloc[i]['Number'])] = input_concat
    return Xnet_features


In [465]:
en_output = get_img_features(test.iloc[0:20,], (224,224,3))

In [466]:
en_output

{2145: array([[4.9550453e-04, 2.0991680e-03, 1.7735970e-03, ..., 8.4199679e-01,
         8.4065425e-01, 6.8691975e-01]], dtype=float32),
 1051: array([[0.        , 0.00184126, 0.00116009, ..., 0.81290716, 0.81862867,
         0.66535187]], dtype=float32),
 1131: array([[4.6907488e-04, 2.1022649e-03, 2.4972956e-03, ..., 8.2906669e-01,
         8.4035921e-01, 6.7798662e-01]], dtype=float32),
 3616: array([[1.4932631e-04, 1.7147649e-03, 2.8314192e-03, ..., 8.4204483e-01,
         8.4104896e-01, 6.8688482e-01]], dtype=float32),
 3339: array([[7.0486767e-06, 2.1787314e-03, 1.8758981e-03, ..., 8.3596873e-01,
         8.4641933e-01, 6.8505216e-01]], dtype=float32),
 369: array([[3.8068678e-04, 1.8055823e-03, 1.5445231e-03, ..., 8.1980544e-01,
         8.2468694e-01, 6.8049926e-01]], dtype=float32),
 1955: array([[3.8213431e-04, 2.0339522e-03, 1.6955610e-03, ..., 8.3482862e-01,
         8.2959968e-01, 6.8235379e-01]], dtype=float32),
 3813: array([[5.5457797e-04, 2.3255087e-03, 1.6916335e-03, 

In [469]:
en_output[2145]

array([[4.9550453e-04, 2.0991680e-03, 1.7735970e-03, ..., 8.4199679e-01,
        8.4065425e-01, 6.8691975e-01]], dtype=float32)

In [470]:
pad_size = 155
BATCH_SIZE = 14


In [472]:
# X_train_img = train["Number"]
# X_cv_img = test["Number"] 
# y_train_rep = train['Clean_capt'] 
# y_cv_rep = test['Clean_capt']

# def load_image(id_, report):
#     '''Loads the Image Features with their corresponding Ids'''
#     img_feature = en_output[id_][0]
#     return img_feature, report

# def create_dataset(img_name_train, report_train):
#     dataset = tf.data.Dataset.from_tensor_slices((img_name_train, report_train))
#   # Use map to load the numpy files in parallel
#     dataset = dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2],
#                           [tf.float32, tf.string]),
#                           num_parallel_calls=tf.data.experimental.AUTOTUNE)
#   # Shuffle and batch
#     dataset = dataset.shuffle(500).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     return dataset

# train_dataset = create_dataset(X_train_img, y_train_rep)
# cv_dataset = create_dataset(X_cv_img, y_cv_rep)

In [475]:
c = test.iloc[0:20,]  ### using all test data instead
X_train_img = c['Number']  
y_train_rep = c["Clean_capt"]

## will need 
# X_test_img = test['Number']  
# y_test_rep = train["Clean_capt"]


In [476]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(y_train_rep.values) 
vocab_size = len(tokenizer.word_index.keys()) + 1

In [477]:
vocab_size

10

In [478]:
c["Clean_capt"]

0                         no acute disease
1         no acute cardiopulmonary disease
2                    negative chest x xxxx
3                        no active disease
4         no acute cardiopulmonary disease
5                     no acute abnormality
6     no acute cardiopulmonary abnormality
7     no acute cardiopulmonary abnormality
8     no acute cardiopulmonary abnormality
9            no acute preoperative finding
10              no acute pulmonary disease
11    no acute cardiopulmonary abnormality
12    no acute cardiopulmonary abnormality
13        no acute cardiopulmonary finding
14              no acute pulmonary disease
15                       no active disease
16    no acute cardiopulmonary abnormality
17            heart size normal lung clear
18    no acute cardiopulmonary abnormality
19        no acute cardiopulmonary finding
Name: Clean_capt, dtype: object

In [479]:
X_train_img

0     2145
1     1051
2     1131
3     3616
4     3339
5      369
6     1955
7     3813
8     2113
9     1473
10     927
11     866
12    1968
13    1324
14    1782
15    2766
16    1569
17    2372
18      36
19    2033
Name: Number, dtype: int64

In [480]:
def load_image(id_, report):
    '''Loads the Image Features with their corresponding Ids'''
    img_feature = en_output[id_][0]
    return img_feature, report

In [481]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_img, y_train_rep))
#cv_dataset = tf.data.Dataset.from_tensor_slices((X_test_img, y_test_rep))


In [482]:
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2],
                          [tf.float32, tf.string]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
# cv_dataset = cv_dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2],
#                           [tf.float32, tf.string]),
#                           num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [483]:
train_dataset = train_dataset.shuffle(500).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
##cv_dataset = cv_dataset.shuffle(500).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [484]:
for a, b in train_dataset:
    print(a)
    print(b)

tf.Tensor(
[[2.0767657e-04 2.0857274e-03 1.1943693e-03 ... 7.4444431e-01
  8.4086668e-01 6.1591518e-01]
 [6.2002109e-05 1.8069228e-03 2.3020096e-03 ... 8.2237083e-01
  8.3359653e-01 6.7047399e-01]
 [0.0000000e+00 1.8412616e-03 1.1600930e-03 ... 8.1290716e-01
  8.1862867e-01 6.6535187e-01]
 ...
 [4.3449336e-06 2.1428221e-03 1.7226979e-03 ... 8.3789164e-01
  8.2778198e-01 6.8506002e-01]
 [0.0000000e+00 1.8209199e-03 2.2260072e-03 ... 8.3009702e-01
  8.6562520e-01 6.7471540e-01]
 [3.5670062e-04 2.2255220e-03 1.7428959e-03 ... 8.2828504e-01
  8.2490283e-01 6.7652863e-01]], shape=(14, 2048), dtype=float32)
tf.Tensor(
[b'no acute cardiopulmonary finding'
 b'no acute cardiopulmonary abnormality'
 b'no acute cardiopulmonary disease' b'no acute preoperative finding'
 b'no active disease' b'no acute cardiopulmonary abnormality'
 b'no acute cardiopulmonary disease' b'no acute pulmonary disease'
 b'no acute cardiopulmonary abnormality' b'no acute abnormality'
 b'heart size normal lung clear' b'no 

In [485]:
input1 = Input(shape=(2048), name='Image_1')
dense1 = Dense(256, kernel_initializer=tf.keras.initializers.glorot_uniform(seed = 56),
               name='dense_encoder')(input1)

input2 = Input(shape=(155), name='Text_Input')
emb_layer = Embedding(input_dim = vocab_size, output_dim = 300, input_length=155, mask_zero=True,
                      trainable=False, name="Embedding_layer") ##weights=[embedding_matrix], 
emb = emb_layer(input2)
LSTM2 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")
LSTM2_output = LSTM2(emb)
dropout1 = Dropout(0.5, name='dropout1')(LSTM2_output)

dec =  tf.keras.layers.Add()([dense1, dropout1])

fc1 = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed = 63),
            name='fc1')
fc1_output = fc1(dec)
output_layer = Dense(vocab_size, activation='softmax', name='Output_layer')
output = output_layer(fc1_output)

encoder_decoder = Model(inputs = [input1, input2], outputs = output)

In [486]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')

def custom_loss(y_true, y_pred):
    #getting mask value to not consider those words which are not present in the true caption
    print(y_true)
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))

    loss_ = loss_function(y_true, y_pred)
    
    #converting mask dtype to loss_ dtype
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    #applying the mask to loss
    loss_ = loss_*mask
    
    #returning mean over all the values
    return tf.reduce_mean(loss_)

In [487]:

optimizer = tf.keras.optimizers.Adam(0.001)
encoder_decoder.compile(optimizer,loss = custom_loss)

In [496]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/train'
val_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

In [488]:
print(encoder_decoder.summary())


Model: "model_56"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Text_Input (InputLayer)        [(None, 155)]        0           []                               
                                                                                                  
 Embedding_layer (Embedding)    (None, 155, 300)     3000        ['Text_Input[0][0]']             
                                                                                                  
 Image_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 LSTM2 (LSTM)                   (None, 256)          570368      ['Embedding_layer[0][0]']        
                                                                                           

In [489]:
def bytes_to_string(arr):
    '''The generator gives provides data in bytes. This function converts them back to strings for manipulation'''
    for i in range(len(arr)):
        arr[i] = arr[i].decode('utf-8')


In [245]:
en_output

{2145: array([[4.9550453e-04, 2.0991680e-03, 1.7735970e-03, ..., 8.4199679e-01,
         8.4065425e-01, 6.8691975e-01]], dtype=float32),
 1051: array([[0.        , 0.00184126, 0.00116009, ..., 0.81290716, 0.81862867,
         0.66535187]], dtype=float32),
 1131: array([[4.6907488e-04, 2.1022649e-03, 2.4972956e-03, ..., 8.2906669e-01,
         8.4035921e-01, 6.7798662e-01]], dtype=float32),
 3616: array([[1.4932631e-04, 1.7147649e-03, 2.8314192e-03, ..., 8.4204483e-01,
         8.4104896e-01, 6.8688482e-01]], dtype=float32),
 3339: array([[7.0486767e-06, 2.1787314e-03, 1.8758981e-03, ..., 8.3596873e-01,
         8.4641933e-01, 6.8505216e-01]], dtype=float32)}

In [490]:
def bytes_to_string(arr):
    '''The generator gives provides data in bytes. This function converts them back to strings for manipulation'''
    for i in range(len(arr)):
        arr[i] = arr[i].decode('utf-8')
    return arr

In [491]:
def convert(images, reports):
    '''This function takes the batch of data and converts them into a new dataset(A WORD BY WORD DATASET)'''
    imgs = []
    in_reports = []
    out_reports = []
    sequence = []
    for i in range(len(images)):
        sequence = [tokenizer.word_index[e] for e in reports[i].split() if e in tokenizer.word_index.keys()]
        for j in range(1,len(sequence)):

            in_seq = sequence[:j]
            out_seq = sequence[j]
            out_seq = tf.keras.utils.to_categorical(out_seq, num_classes=vocab_size)

            imgs.append(images[i])
          #  print(in_seq)
            in_reports.append(in_seq)
           # print(out_seq)
            out_reports.append(out_seq)
        
    return np.array(imgs), np.array(in_reports), np.array(out_reports)


In [492]:
cv_dataset = train_dataset
##if have cv_dataset then comment

In [501]:
epoch_train_loss = []
epoch_val_loss = []
EPOCH = 10

for epoch in range(EPOCH):
    print('EPOCH : ',epoch+1)
    batch_loss_tr = 0
    batch_loss_vl = 0
    start = time.time()
    for img, report in train_dataset:
       
        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=155, padding='post')
        
        results = encoder_decoder.train_on_batch([img_input, rep_input], output_word)
        
        batch_loss_tr += results
    print(X_train_img.shape[0])
    train_loss = batch_loss_tr/(X_train_img.shape[0]//BATCH_SIZE)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss, step = epoch)
    
    for img, report in cv_dataset:
        
        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=155, padding='post')
        results = encoder_decoder.test_on_batch([img_input, rep_input], output_word)
        batch_loss_vl += results
        
    val_loss = batch_loss_vl/(X_cv_img.shape[0]//14)
    with val_summary_writer.as_default():
        tf.summary.scalar('loss', val_loss, step = epoch)

    epoch_train_loss.append(train_loss)
    epoch_val_loss.append(val_loss)
    
    print('Training Loss: {},  Val Loss: {}'.format(train_loss, val_loss))
    print('Time Taken for this Epoch : {} sec'.format(time.time()-start))   
    encoder_decoder.save_weights('weights/BM7_new_model1_epoch_'+ str(epoch+1) + '.h5')

EPOCH :  1
20
Training Loss: 0.3524944335222244,  Val Loss: 0.005373527606328328
Time Taken for this Epoch : 0.6853089332580566 sec
EPOCH :  2
20
Training Loss: 0.3472699075937271,  Val Loss: 0.006023310517009936
Time Taken for this Epoch : 0.5130889415740967 sec
EPOCH :  3
20
Training Loss: 0.3615592122077942,  Val Loss: 0.005815087964660243
Time Taken for this Epoch : 0.4712870121002197 sec
EPOCH :  4
20
Training Loss: 0.29299361258745193,  Val Loss: 0.005489720587144818
Time Taken for this Epoch : 0.5298151969909668 sec
EPOCH :  5
20
Training Loss: 0.2851252108812332,  Val Loss: 0.004782014117951979
Time Taken for this Epoch : 0.7960128784179688 sec
EPOCH :  6
20
Training Loss: 0.2732304558157921,  Val Loss: 0.005046719783230832
Time Taken for this Epoch : 0.5855510234832764 sec
EPOCH :  7
20
Training Loss: 0.27072934806346893,  Val Loss: 0.004308307902854786
Time Taken for this Epoch : 0.4744250774383545 sec
EPOCH :  8
20
Training Loss: 0.27688272297382355,  Val Loss: 0.00444107502

0